In [0]:
import pyspark.sql.functions as sf
from pyspark.sql.types import *
from time import time
spark.conf.set("spark.sql.session.timeZone", "America/New_York")

In [0]:
# set time window for filtering data
# 1209600 = 2 weeks
unix_time_window = time() - 1209600

In [0]:
df_skill_inputs_enriched = spark.read.table("runescape.02_silver.skill_action_inputs_enriched")

In [0]:

df_skill_xp_enriched = spark.read.table("runescape.02_silver.skill_action_xp_enriched")

In [0]:
# Read data from 'runescape.02_silver.item_mapping'
df_map = spark.read.table("runescape.02_silver.item_mapping")
df_map.display()

In [0]:
df_skill_xp_enriched.display()
df_skill_inputs_enriched.display()

In [0]:
# Read data from 'runescape.01_bronze.1h_prices'
df_hourly = spark.read.table("runescape.02_silver.1h_prices_cleansed").filter(f"time >= {unix_time_window}")
df_hourly.count()
df_hourly.display()

In [0]:
df_hourly = df_hourly.join(df_map, "id")

In [0]:
df_hourly.display()

In [0]:
# Test with making ruby amulets
id_filter = "ID IN ('1679','1603','1619','2357','444')"
id_filter = "ID IN ('1679','1603','1619')" #without gold bar/ gold ore
df_hourly_test = df_hourly.filter(id_filter)
df_hourly_test.display()


Databricks visualization. Run in Databricks to view.

In [0]:
%skip
@udf(DoubleType())
def get_profit(time, price, id, input_qty)
    return profit

In [0]:
%skip
df_hourly_combined = df_hourly.join(df_skill_inputs_enriched, 'id')
id_filter = "ID IN ('1679','1603','1619','2357','444')"
df_hourly_combined = df_hourly_combined.filter(id_filter)
df_hourly_combined.display()

In [0]:
df_hourly.display()
df_skill_inputs_enriched.display()

In [0]:
df_input_cost = df_hourly.join(df_skill_inputs_enriched, df_hourly.id == df_skill_inputs_enriched.id_input)
#id_filter = "id_output IN ('1679','1603','1619','2357','444')"
#df_input_cost = df_input_cost.filter(id_filter)
df_input_cost.display()

In [0]:
# The above can be used with an summation to get the input cost
df_input_total_cost = df_input_cost.groupBy("time","output", "skill").sum("avg1HourHigh", "avg1HourLow")
df_input_total_cost = df_input_total_cost.withColumnsRenamed({"sum(avg1HourHigh)": "total_input_cost_high"
                                                              , "sum(avg1HourLow)": "total_input_cost_low"})

df_input_total_cost.display()

Databricks visualization. Run in Databricks to view.

In [0]:
df_profit = df_input_total_cost.join(df_hourly, [df_hourly.time == df_input_total_cost.time, df_hourly.name == df_input_total_cost.output])
# not taxed
df_profit = df_profit.withColumn("low_profit", sf.col("avg1HourHigh") - sf.col("total_input_cost_high"))
# not taxed
df_profit = df_profit.withColumn("high_profit", sf.col("avg1HourLow") - sf.col("total_input_cost_low"))
df_profit.display()

Databricks visualization. Run in Databricks to view.

In [0]:
df_profit_ruby_unstrung = df_profit.filter("output == 'Ruby amulet (u)'")
df_profit_ruby_unstrung.display()

Databricks visualization. Run in Databricks to view.

In [0]:
df_profit_all = df_profit
df_profit_all.display()


Databricks visualization. Run in Databricks to view.